In [2]:
import torch
from torchvision.models import AlexNet
import torch.nn as nn
from torchviz import make_dot
from collections import OrderedDict
from functools import partial
from itertools import repeat
import collections.abc
#Trasnforemr相关组件====================================
def _ntuple(n):
    def parse(x):
        if isinstance(x, collections.abc.Iterable):
            return x
        return tuple(repeat(x, n))

    return parse
to_2tuple = _ntuple(2)

class PatchEmbed(nn.Module):#默认为vit—base
    # patch_size=16,
    # embed_dim=768,
    # depth=12,
    # num_heads=12,
    # mlp_ratio=4,

    def __init__(self,In_Channels:int=1,Out_Channels:int=768,img_size:int=224,patch_size:int=16):
        super(PatchEmbed,self).__init__()
        self.proj=nn.Conv2d(in_channels=In_Channels,out_channels=Out_Channels, kernel_size=patch_size,stride=patch_size,)
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size = (img_size[0] // patch_size[0],
                          img_size[1] // patch_size[1])
        self.num_patches = self.grid_size[0] * self.grid_size[1]
    def forward(self,x):
        print(x.shape)
        x = self.proj(x).flatten(2).transpose(1, 2)
        print("Out shale:",x.shape)
        return x
def Test_PatchEmbed():
    model=PatchEmbed(In_Channels=1,Out_Channels=384)
    output=model(torch.rand(1,1,224,224))
    print(output.shape)



In [3]:
#构建Embedding 层
In_Channels=3
Batch_Size=2
Out_Channels=768
Picture_Size=224
x=torch.rand(Batch_Size,In_Channels,Picture_Size,Picture_Size)
Embedding_Layer=PatchEmbed(In_Channels=In_Channels,Out_Channels=Out_Channels)
Embedding_Out=Embedding_Layer(x)

torch.Size([2, 3, 224, 224])
Out shale: torch.Size([2, 196, 768])


In [4]:
#开始构建Encoder
#[B,H,W]
B=Embedding_Out.shape[0]
H=Embedding_Out.shape[1]
W=Embedding_Out.shape[2]
Num_Heads=8

Single_Encoder = nn.TransformerEncoderLayer(d_model=W, nhead=Num_Heads,batch_first=True,norm_first=True)#在Vit中的norm需要位于atten和FF之前
Encoders= nn.TransformerEncoder(Single_Encoder,num_layers=12,norm=None)#构建多个连在一起的Encoder
                                                                      #nomrm 就是在末尾加一个normlization

Encoder_Out=Encoders(Embedding_Out)
print(Encoder_Out.shape)
                                            

# print(x.shape)
# print(Single_Encoder)

torch.Size([2, 196, 768])


In [23]:
NormLayer_After_Encoder=nn.LayerNorm(768,)
Norm_Out=NormLayer_After_Encoder(Encoder_Out)
print(Norm_Out.shape)
print(NormLayer_After_Encoder.weight.shape)
Norm_Out=Norm_Out[:,0]
print(Norm_Out.shape)

torch.Size([2, 196, 768])
torch.Size([768])
torch.Size([2, 768])


In [7]:
#开始构建Mlp head,Fq-vit直接放一个linear就没了，，，，
Num_Class=1
Mlp_Head=nn.Linear(W,Num_Class)
Class_Out=Mlp_Head(Encoder_Out)
print(Class_Out.shape)

torch.Size([2, 196, 1])
